# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [32]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [33]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,rongelap,11.1635,166.8941,28.48,77,36,5.73,MH,1689491254
1,1,ilhabela,-23.7781,-45.3581,16.68,85,0,1.81,BR,1689491255
2,2,griffith,41.5284,-87.4237,21.05,91,0,2.06,US,1689491255
3,3,tofol,5.3248,163.0078,25.88,94,100,1.54,FM,1689491255
4,4,adamstown,-25.0660,-130.1015,21.11,88,100,5.07,PN,1689491255


---
### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [34]:
%%capture --no-display

# Configure the map plot

map_city = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    tiles="OSM",
    frame_width=600,
    frame_height=400,
    size="Humidity",
    scale=0.5,
    color="City"
)
# Display the map
map_city


:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [35]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_cities = city_data_df[
    (city_data_df["Max Temp"] < 25) &
    (city_data_df["Max Temp"] > 20) &
    (city_data_df["Wind Speed"] < 10) &
    (city_data_df["Cloudiness"] == 0)
].copy()

# Drop any rows with null values
ideal_cities.dropna(inplace=True)

# Display sample data
ideal_cities.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
2,2,griffith,41.5284,-87.4237,21.05,91,0,2.06,US,1689491255
22,22,anadyr,64.7500,177.4833,21.28,49,0,7.00,RU,1689491260
78,78,douglas,41.3000,-96.1670,21.48,90,0,0.00,US,1689491275
88,88,remire-montjoly,4.9167,-52.2667,23.02,100,0,0.00,GF,1689491278
112,112,santa ana,33.7456,-117.8678,24.29,85,0,1.54,US,1689491288


### Step 3: Create a new DataFrame called `hotel_df`.

In [36]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_cities[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
2,griffith,US,41.5284,-87.4237,91,
22,anadyr,RU,64.7500,177.4833,49,
78,douglas,US,41.3000,-96.1670,90,
88,remire-montjoly,GF,4.9167,-52.2667,100,
112,santa ana,US,33.7456,-117.8678,85,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [37]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories":"accommodation.hotel",
    "apiKey": geoapify_key,
    "limit":20
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lon = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lon},{lat},{radius}"
    params["bias"] = f"proximity:{lon},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
griffith - nearest hotel: Quality Inn & Suites
anadyr - nearest hotel: Гостевой дом
douglas - nearest hotel: Residence Inn Omaha West
remire-montjoly - nearest hotel: Complexe Belova
santa ana - nearest hotel: Holiday Inn Express
quelimane - nearest hotel: Mabassa
silver city - nearest hotel: Murray Hotel
tsaratanana - nearest hotel: No hotel found
carnarvon - nearest hotel: No hotel found
twin falls - nearest hotel: No hotel found
geraldton - nearest hotel: Ocean Centre Hotel
mena - nearest hotel: Континент
luanda - nearest hotel: Hotel Forum
namibe - nearest hotel: Hotel Chik Chik Namibe
panaba - nearest hotel: No hotel found
portland - nearest hotel: the Hoxton
ankazoabo - nearest hotel: No hotel found
holualoa - nearest hotel: Kona Hotel
mead - nearest hotel: Best Western Plus Spokane North
karasu - nearest hotel: No hotel found
ampanihy - nearest hotel: Hotel restaurant Angora
kailua-kona - nearest hotel: Kona Seaside Hotel
papao - nearest hotel: Hiti Moana V

,City,Country,Lat,Lng,Humidity,Hotel Name
2,griffith,US,41.5284,-87.4237,91,Quality Inn & Suites
22,anadyr,RU,64.7500,177.4833,49,Гостевой дом
78,douglas,US,41.3000,-96.1670,90,Residence Inn Omaha West
88,remire-montjoly,GF,4.9167,-52.2667,100,Complexe Belova
112,santa ana,US,33.7456,-117.8678,85,Holiday Inn Express
115,quelimane,MZ,-17.8786,36.8883,66,Mabassa
163,silver city,US,32.7701,-108.2803,62,Murray Hotel
190,tsaratanana,MG,-16.7833,47.6500,50,No hotel found
229,carnarvon,AU,-24.8667,113.6333,35,No hotel found
257,twin falls,US,42.3505,-114.6445,35,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [38]:
%%capture --no-display
# Configure the map plot
map_city = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    tiles="OSM",
    frame_width=600,
    frame_height=400,
    size="Humidity",
    color="City",
    hover_cols=["City", "Country", "Hotel Name"]
)

# Display the map
map_city


:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)